In [1]:
# 初始化环境
! pip install openai

     -------------------------------------- 70.1/70.1 kB 424.3 kB/s eta 0:00:00
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
     ---------------------------------------- 77.1/77.1 kB 1.1 MB/s eta 0:00:00
  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)
  Using cached charset_normalizer-3.0.1-cp310-cp310-win_amd64.whl (96 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp310-cp310-win_amd64.whl (28 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-win_amd64.whl (33 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached yarl-1.8.2-cp310-cp310-win_amd64.whl (56 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)


# Let's Step by Step
先准备一个[openai的账户](https://platform.openai.com/account/api-keys)，注册之后有送的额度，并且不限制ip地址。具体文档可以参考[openai的doc](https://platform.openai.com/docs/models)

In [6]:
import openai
OPENAI_KEY = "XXXXXX"
openai.api_key = OPENAI_KEY

准备一些简单的脚手架，language model本质上是个续写器，你给他个开头，他给你接着写下去，`text-davinci-002`是弱化版的gpt-3.5，`gpt-3.5-turbo`是效率优化版的chatgpt，`code-davinci-002`是弱化版的`codex`

In [204]:
def ask_gpt_3(text, max_tokens=256):
    response = openai.Completion.create(
            model="text-davinci-002", prompt=text, temperature=0,
            max_tokens=max_tokens, top_p=1, frequency_penalty=0, presence_penalty=0 )
    return response["choices"][0]["text"]

def ask_chatgpt(text, system_content="", max_tokens=100):
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": text}
                ],

            temperature=0
        )
    return response["choices"][0]["message"]["content"]

def ask_codex(text, max_tokens=512):
    response = openai.Completion.create(
            model="code-davinci-002", prompt=text, temperature=0, max_tokens=max_tokens,
            top_p=1, frequency_penalty=0, presence_penalty=0)
    return response["choices"][0]["text"]

然后我们准备一个简单的小学数学习题，然后分别请教下chatgpt和gpt-3（从这以后的结果因为有随机性，因此每次执行的结果可能不太一样，其实gpt3的中文能力还是比英文差不少，不过为了有意思，尽量用了中文，而且prompt我个人没仔细优化，因此可能可复现性一般，而且为了证明效果，我使用了效果比较差的text-davinci-002的版本）。

In [130]:
qa_1 = """jack has $35. She bought 5 apple for $6 each. How much money does he have left?
a:"""
print(ask_gpt_3(qa_1))

 Jack has $11 left.


这个结果好像不太对啊，没事，让我们安慰一下gpt，你可以的，别紧张，慢慢来

In [131]:
qa_1 = """jack has $35. She bought 5 apple for $6 each. How much money does he have left?
a: let's calc it step by step, """
print(ask_gpt_3(qa_1))


1) first we need to find out how much money she spent on apples, 
2) then we need to subtract that amount from the total amount of money she had 
3) the answer we get will be the amount of money she has left

1) she bought 5 apples for $6 each, so she spent $30 on apples
2) $35 - $30 = $5
3) she has $5 left


这次果然就对了，看来鼓励是有用的，或者我们换个方式，当作一个老师，教他怎么算

In [128]:
qa_1_new = """q: Olivia has $23. She bought 5 bagels for $3 each. How much money does she have left?
a: let's step by step, Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. Olivia has $2 left.
q: jack has $35. She bought 5 apple for $6 each. How much money does he have left? 
a: let's step by step, """
print(ask_gpt_3(qa_1_new))

==========================GPT3=============
 Jack had 35 dollars. 5 apples for 6 dollars each will be 5 x 6 = 30 dollars. So he has 35 - 30 dollars left. 35 - 30 is 5. Jack has $5 left.


这就很像平时学习了，课上讲个习题，然后老师出个新题来给你做，最后我们看看升级版的chatgpt的答案

In [129]:
print("=====================ChatGPT=============")
print(ask_chatgpt(qa_1))

=====================ChatGPT=============


First, we need to find out how much money Jack spent on the apples. 

5 apples at $6 each is: 

5 x $6 = $30 

So Jack spent $30 on the apples. 

To find out how much money he has left, we need to subtract the amount he spent from the amount he started with: 

$35 - $30 = $5 

Therefore, Jack has $5 left after buying 5 apples for $6 each.


chatgpt还是比较强，直接就把思路给出来了。

这里，我们拼在前面给模型输入的内容叫做prompt

前面的前两种方式，通过给模型一些例子，引导模型进行分步思考或者按照特定方式思考，这就是CoT(Chain of Thought)

# CoT

CoT这个概念一开始是由google brain的[Chain-of-Thought Prompting Elicits Reasoning in Large Language Models
](https://arxiv.org/abs/2201.11903v6)这篇论文提出来的概念，原文的定义是`a series of intermediate reasoning
steps`，即上文例子中的做题的推理过程。只提供`let's step by step`这种引导的句子，没有样例的情况成为`zero shot CoT`，而第二个有样例(数量其实不限制)的这种输入称为`few show CoT`。这篇文章中认为CoT可以有一些明显的优势

1. 使用CoT可以让模型分解出更多的中间步骤，从而对更难的问题使用更多额外的计算量。 （这个很容易理解，毕竟生产的句子都更长了）
2. CoT使得模型提供了更多的可解释性。 （有了中间思路，更容易理解了）
3. CoT可是用于更多使用语言描述的场景，而非仅仅在数学、符号推理类的场景。
4. 可以将LLM应用在`few shot`的场景。（第二个例子的样子）

这里对第三个场景举个例子，给GPT一点阴阳怪气的震撼

In [141]:
qa_2_n = """q: 都下班了你才把文档给我，我真是谢谢你!。判断上句是正面评价还是负面评价?
a:"""
print(ask_gpt_3(qa_2_n))

 正面评价


还是太年轻了，我们引导下

In [142]:
qa_a_n_zero = """q: 都下班了你才把文档给我，我真是谢谢你!。判断上句是正面评价还是负面评价?
a: 让我们一点点来分析，"""
print(ask_gpt_3(qa_a_n_zero))

“都下班了你才把文档给我”，说明你的工作时间不够充分，这是负面评价。“我真是谢谢你”，说明你对对方的帮助表示感谢，这是正面评价。所以上句是负面评价。


最终结果对了，但是没看出来阴阳怪气

In [144]:
qa_a_n_few_1 = """q: 我都已经睡觉了你来通知我，我真的爱死你了！
a: 让我们一点点来分析，前半句“我都已经睡觉了你来通知我”说明我很不情愿，所以后面的“我真的爱死你了”是表示讽刺，所以是负面评价。
q: 都下班了你才把文档给我，我真是谢谢你!。判断上句是正面评价还是负面评价?
a: 让我们一点点来分析"""
print(ask_gpt_3(qa_a_n_few_1))

，前半句“都下班了你才把文档给我”说明我很不情愿，所以后面的“我真是谢谢你!”是表示讽刺，所以是负面评价。


学习到了阴阳怪气，让我看看他别被带偏了

In [145]:
qa_a_n_few_2 = """q: 我都已经睡觉了你来通知我，我真的爱死你了！
a: 让我们一点点来分析，前半句“我都已经睡觉了你来通知我”说明我很不情愿，所以后面的“我真的爱死你了”是表示讽刺，所以是负面评价。
q: 帮我搬运了这么多东西，真的是谢谢你!。判断上句是正面评价还是负面评价?
a: 让我们一点点来分析"""
print(ask_gpt_3(qa_a_n_few_2))

，前半句“帮我搬运了这么多东西”说明我很感激，所以后面的“真的是谢谢你!”是表示正面评价。


这里可以看出来，提供的gpt3可以从prompt中学到抽象“反讽”的意义，并且没有带偏。也证明了第三点，基于自然语言的任务，非数学、符号推导类的场景，已经可以适用于CoT的能力。

另一个现象，如下图所示，以前在GPT3的paper中提到大模型在一些问题上与出现`flat scaling curve`的现象（即模型性能表面上并没有随着参数量的增加而增加），在CoT prompt中没有出现。

![](./imgs/cot_qa.PNG)

当然这里有很多玄学，prompt的构造会影响性能，比如下面这种情况，（你自己有没有注意苹果能浮起来吗？）

In [170]:
print(ask_gpt_3("""q: 苹果会浮在水面上吗？
a: """))

print("============== 我是分割线================")

print(ask_gpt_3("""q: 苹果会浮在水面上吗？
a: 让我们一点点来分析，"""))

print("============== 我是分割线================")

print(ask_gpt_3("""q: 苹果会浮在水面上吗？
a: 让我们一步步来思考，"""))

print("============== 我是分割线================")
print(ask_chatgpt("""q: 苹果会浮在水面上吗？
a: """))

print("============== 我是分割线================")
print(ask_chatgpt("苹果能浮在水里吗？"))


会的，苹果是浮力大于它的重力的。
============== 我是分割线================
苹果的密度是0.7g/cm^3，水的密度是1g/cm^3，所以苹果会浮在水面上。
============== 我是分割线================
首先，苹果是一种水果，水果里面含有水分，所以苹果会吸收水分。其次，苹果的密度小于水，所以苹果会浮在水面上。
============== 我是分割线================
是的，苹果会浮在水面上。这是因为苹果的密度比水的密度小，所以它会浮在水面上。
============== 我是分割线================


苹果本身是比水密度大的，因此它不能浮在水里。不过，如果苹果被切成小块或碎片，它们可能会浮在水面上，因为它们的密度会变得比水小。


prompt有一点变化，给的思路变化都很大。

此外，[zeroshot CoT的论文](https://arxiv.org/pdf/2205.11916.pdf)里面也展示作者玄学的尝试

![](./imgs/zero_cot.PNG)

### 还能更玄学一点

In [198]:
qa_ltm_2 = """q: 蓄水池进水到装满需要30分钟，蓄水池装满放水到放空需要15分钟，装满的蓄水池一边进水一边放水，需要多久放空?
a: 让我们一步步来，"""

print(ask_gpt_3(qa_ltm_2))

蓄水池装满需要30分钟，蓄水池放空需要15分钟，所以如果蓄水池一边进水一边放水，那么总共需要45分钟。


我们让他自己分解一下问题

In [188]:
qa_ltm_3 = """q: 蓄水池进水到装满需要30分钟，蓄水池装满放水到放空需要15分钟，装满的蓄水池一边进水一边放水，需要多久放空?
a: 为了计算"需要多久放空"，我们需要先计算" """

print(ask_gpt_3(qa_ltm_3))

一边进水一边放水"的速度。

蓄水池进水的速度是每分钟2升，放水的速度是每分钟3升。所以，一边进水一边放水的速度是每分钟5升。

所以，如果一边进水一边放水，需要多久放空？

需要多久放空取决于蓄水池的大小。如果蓄水池是100升的，那么需要多久放空？



然后把第一行的满足template的拼进来

In [195]:
qa_ltm_4 = """q: 蓄水池进水到装满需要30分钟，蓄水池装满放水到放空需要15分钟，装满的蓄水池一边进水一边放水，"一边进水一边放水"的速度是多少?
a: """
print(ask_gpt_3(qa_ltm_4))

一边进水一边放水的速度是2分钟/升。


再把他自己的答案拼进问题

In [197]:
qa_ltm_5 = """q: 蓄水池进水到装满需要30分钟，蓄水池装满放水到放空需要15分钟，装满的蓄水池一边进水一边放水，一边进水一边放水的速度是2分钟/升，需要多久放空?
a: """
print(ask_gpt_3(qa_ltm_5))



30分钟


这种引导他自己分解问题，然后再回答问题的方式就叫做[Least-to-Most Prompting](https://arxiv.org/pdf/2205.10625v2.pdf)

此外还有很玄学的[Auto CoT](https://arxiv.org/abs/2210.03493)，自己生成`few shot`的prompt，然后带着自己做题

![](./imgs/auto_cot.PNG)

还有[Program of Thoughts](https://arxiv.org/pdf/2211.12588v3.pdf)

In [205]:
pot_q = """# question: jack has $35. She bought 5 apple for $6 each. How much money does he have left?
# answer this question by implement solver function in python
def solver():
    # let's write a python program step by step and then return the answer
    # firstly, we need define following variables
"""
print(ask_codex(pot_q))

    # jack's money
    jack_money = 35
    # apple's price
    apple_price = 6
    # apple's number
    apple_number = 5
    # jack's money left
    jack_money_left = jack_money - apple_price * apple_number
    # return the answer
    return jack_money_left

# print the answer
print(solver())

# question: jack has $35. He bought 5 apple for $6 each. How much money does he have left?
# answer this question by implement solver function in python
def solver():
    # let's write a python program step by step and then return the answer
    # firstly, we need define following variables
    # jack's money
    jack_money = 35
    # apple's price
    apple_price = 6
    # apple's number
    apple_number = 5
    # jack's money left
    jack_money_left = jack_money - apple_price * apple_number
    # return the answer
    return jack_money_left

# print the answer
print(solver())

# question: jack has $35. He bought 5 apple for $6 each. How much money does he have left?
# answer this question by i

除了单纯的文本以及代码之外，还有结合图片的[MM-CoT](https://arxiv.org/abs/2302.00923)，通过结合多模态信息，仅有0.2B的模型参数量下，依旧由于GPT-3.5的效果。

![](./imgs/mmcot.PNG)

虽然试了很多方法，其实还是有很多问题

- CoT的能力到底是怎么获得的？
- 多大的参数量下，才能体现基本的CoT的能力？
  

最后，既然CoT效果这么好，我们不能用来继续优化pretrain的模型吗？这就引出了接下来的第二部分instruction finetuning